In [51]:
from elasticsearch import Elasticsearch
import pandas as pd

In [52]:
es = Elasticsearch(
    "https://localhost:9200",
    basic_auth=("elastic", "password"),
        ca_certs="path to http_ca.crt"
)

es.ping()

True

## Preparing the Data


In [53]:
brand_category_df = pd.read_csv('brand_category.csv')
categories_df = pd.read_csv('categories.csv')
offer_retailer_df = pd.read_csv('offer_retailer.csv')

In [54]:
brand_category_df.fillna("None", inplace=True)
categories_df.fillna("None", inplace=True)
offer_retailer_df.fillna("None", inplace=True)

In [55]:
df = pd.merge(offer_retailer_df, brand_category_df, on='BRAND', how='left')
df.head()


,OFFER,RETAILER,BRAND,CATEGORY,RECEIPTS
0,Spend $50 on a Full-Priced new Club Membership,SAMS CLUB,SAMS CLUB,NaN,NaN
1,"Beyond Meat® Plant-Based products, spend $25",None,BEYOND MEAT,Plant-Based Meat,1584.0
2,"Beyond Meat® Plant-Based products, spend $25",None,BEYOND MEAT,Frozen Plant-Based Meat,313.0
3,"Beyond Meat® Plant-Based products, spend $25",None,BEYOND MEAT,Packaged Meat,30.0
4,Good Humor Viennetta Frozen Vanilla Cake,None,GOOD HUMOR,Frozen Desserts,1052.0


In [56]:
df = pd.merge(df, categories_df, on='CATEGORY', how='left')
df.head()

,OFFER,RETAILER,BRAND,CATEGORY,RECEIPTS,CATEGORY_ID,IS_CHILD_CATEGORY_TO
0,Spend $50 on a Full-Priced new Club Membership,SAMS CLUB,SAMS CLUB,NaN,NaN,NaN,NaN
1,"Beyond Meat® Plant-Based products, spend $25",None,BEYOND MEAT,Plant-Based Meat,1584.0,d8cb60e5-b0c6-478a-971d-c6c55b17831f,Meat & Seafood
2,"Beyond Meat® Plant-Based products, spend $25",None,BEYOND MEAT,Frozen Plant-Based Meat,313.0,8e0a9431-5462-4810-9f65-68fe36adf454,Frozen
3,"Beyond Meat® Plant-Based products, spend $25",None,BEYOND MEAT,Packaged Meat,30.0,e73f7957-0e65-4466-9588-795bdc5f67ac,Pantry
4,Good Humor Viennetta Frozen Vanilla Cake,None,GOOD HUMOR,Frozen Desserts,1052.0,38160828-1029-4505-9849-673773c5fad3,Frozen


In [57]:
df.fillna("None", inplace=True)

/var/folders/_r/rbpgvk997kx9h0kb4vvfpd3m0000gn/T/ipykernel_45106/3777244692.py:1: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'None' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.fillna("None", inplace=True)


In [58]:
df.head()

,OFFER,RETAILER,BRAND,CATEGORY,RECEIPTS,CATEGORY_ID,IS_CHILD_CATEGORY_TO
0,Spend $50 on a Full-Priced new Club Membership,SAMS CLUB,SAMS CLUB,None,None,None,None
1,"Beyond Meat® Plant-Based products, spend $25",None,BEYOND MEAT,Plant-Based Meat,1584.0,d8cb60e5-b0c6-478a-971d-c6c55b17831f,Meat & Seafood
2,"Beyond Meat® Plant-Based products, spend $25",None,BEYOND MEAT,Frozen Plant-Based Meat,313.0,8e0a9431-5462-4810-9f65-68fe36adf454,Frozen
3,"Beyond Meat® Plant-Based products, spend $25",None,BEYOND MEAT,Packaged Meat,30.0,e73f7957-0e65-4466-9588-795bdc5f67ac,Pantry
4,Good Humor Viennetta Frozen Vanilla Cake,None,GOOD HUMOR,Frozen Desserts,1052.0,38160828-1029-4505-9849-673773c5fad3,Frozen


In [59]:
df.isna().sum()

OFFER                   0
RETAILER                0
BRAND                   0
CATEGORY                0
RECEIPTS                0
CATEGORY_ID             0
IS_CHILD_CATEGORY_TO    0
dtype: int64

## Converting the relevant Fields to Vector using BERT model

In [60]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('all-mpnet-base-v2')

In [62]:
df["BRANDVECTOR"] = df["BRAND"].apply(lambda x: model.encode(x))
df["CATEGORYVECTOR"] = df["CATEGORY"].apply(lambda x: model.encode(x))
df["RETAILERVECTOR"] = df["RETAILER"].apply(lambda x: model.encode(x))

In [63]:
df.head()

,OFFER,RETAILER,BRAND,CATEGORY,RECEIPTS,CATEGORY_ID,IS_CHILD_CATEGORY_TO,BRANDVECTOR,CATEGORYVECTOR,RETAILERVECTOR
0,Spend $50 on a Full-Priced new Club Membership,SAMS CLUB,SAMS CLUB,None,None,None,None,"[0.03979099, -0.034983505, -0.008294199, -0.06...","[0.0068389922, 0.017649423, 0.014443061, 0.008...","[0.03979099, -0.034983505, -0.008294199, -0.06..."
1,"Beyond Meat® Plant-Based products, spend $25",None,BEYOND MEAT,Plant-Based Meat,1584.0,d8cb60e5-b0c6-478a-971d-c6c55b17831f,Meat & Seafood,"[0.041429278, 0.10506146, 0.015181328, -0.0429...","[0.021978693, 0.06316671, -0.0020582196, -0.00...","[0.0068389922, 0.017649423, 0.014443061, 0.008..."
2,"Beyond Meat® Plant-Based products, spend $25",None,BEYOND MEAT,Frozen Plant-Based Meat,313.0,8e0a9431-5462-4810-9f65-68fe36adf454,Frozen,"[0.041429278, 0.10506146, 0.015181328, -0.0429...","[0.025395662, 0.069052935, -0.012121743, 0.021...","[0.0068389922, 0.017649423, 0.014443061, 0.008..."
3,"Beyond Meat® Plant-Based products, spend $25",None,BEYOND MEAT,Packaged Meat,30.0,e73f7957-0e65-4466-9588-795bdc5f67ac,Pantry,"[0.041429278, 0.10506146, 0.015181328, -0.0429...","[0.012175618, 0.051638093, 0.019421726, -0.032...","[0.0068389922, 0.017649423, 0.014443061, 0.008..."
4,Good Humor Viennetta Frozen Vanilla Cake,None,GOOD HUMOR,Frozen Desserts,1052.0,38160828-1029-4505-9849-673773c5fad3,Frozen,"[0.031136386, 0.020597655, -0.02058995, 0.0359...","[0.052119132, -0.0043649375, -0.035171624, 0.0...","[0.0068389922, 0.017649423, 0.014443061, 0.008..."


In [64]:
df.count()

OFFER                   826
RETAILER                826
BRAND                   826
CATEGORY                826
RECEIPTS                826
CATEGORY_ID             826
IS_CHILD_CATEGORY_TO    826
BRANDVECTOR             826
CATEGORYVECTOR          826
RETAILERVECTOR          826
dtype: int64

## creating a new index in elasticsearch

In [92]:
indexMapping = {
    "properties": {
        "Offer": {
            "type": "text"
        },
        "Brand": {
            "type": "text"
        },
        "Category": {
            "type": "text"
        },
        "Retailer": {
            "type": "text"
        },
        "BRANDVECTOR": {
            "type": "dense_vector",
            "dims": 768, 
            "index":True, 
            "similarity": "l2_norm"
        },
        "CATEGORYVECTOR": {
            "type": "dense_vector",
            "dims": 768, 
            "index":True, 
            "similarity": "l2_norm"
        },
        "RETAILERVECTOR": {
            "type": "dense_vector", 
            "dims": 768, 
            "index":True, 
            "similarity": "l2_norm"
        }
    }
}

es.indices.create(index="offer_search", mappings=indexMapping)


ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'offer_search'})

## Ingesting the data into index (Converting tabular format to json)

In [93]:
record_list = df.to_dict(orient='records')
record_list[0]

{'OFFER': 'Spend $50 on a Full-Priced new Club Membership',
 'RETAILER': 'SAMS CLUB',
 'BRAND': 'SAMS CLUB',
 'CATEGORY': 'None',
 'RECEIPTS': 'None',
 'CATEGORY_ID': 'None',
 'IS_CHILD_CATEGORY_TO': 'None',
 'BRANDVECTOR': array([ 3.97909917e-02, -3.49835046e-02, -8.29419866e-03, -6.33950680e-02,
         5.83102033e-02, -5.61157875e-02,  2.37907209e-02, -3.22424434e-02,
        -5.17435595e-02,  2.73093507e-02,  3.46731301e-03,  1.38463592e-02,
         4.59022596e-02, -2.42564315e-03, -8.80824518e-04,  5.45713231e-02,
         4.43143509e-02,  1.26618166e-02, -5.08162528e-02, -3.32144126e-02,
         4.91704908e-04,  3.56912031e-04, -1.16921775e-02,  2.16999091e-02,
        -5.05552068e-03, -2.71723396e-03, -4.14214842e-03,  1.37015944e-02,
        -2.05904478e-04, -2.56131124e-02,  5.86304702e-02, -5.70887029e-02,
         9.58990958e-03,  3.46022360e-02,  1.51384427e-06,  1.23917097e-02,
        -3.96475345e-02, -4.29797433e-02, -2.85501480e-02,  4.39194627e-02,
        -3.949962

### Index each record to offer_search index in elasticsearch

In [94]:
for record in df.to_dict("records"):
    try:
        es.index(index="offer_search", document=record)
    except Exception as e:
        print(e)

# Verify indexed documents count
print(es.count(index="offer_search"))

{'count': 823, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}}


## Data Querying

In [95]:
input_query = "target"
vector_of_input_query = model.encode(input_query)

# Perform KNN search for BrandVector
brand_query = {
    "field": "BRANDVECTOR",
    "query_vector": vector_of_input_query,
    "k": 2,
    "num_candidates": 1000,
}
brand_hits = es.knn_search(index="offer_search", knn=brand_query, _source=["OFFER", 'BRAND'])["hits"]["hits"]

# Perform KNN search for CategoryVector
category_query = {
    "field": "CATEGORYVECTOR",
    "query_vector": vector_of_input_query,
    "k": 2,
    "num_candidates": 1000,
}
category_hits = es.knn_search(index="offer_search", knn=category_query, _source=["OFFER","CATEGORY"])["hits"]["hits"]

# Perform KNN search for RetailerVector
retailer_query = {
    "field": "RETAILERVECTOR",
    "query_vector": vector_of_input_query,
    "k": 2,
    "num_candidates": 1000,
}
retailer_hits = es.knn_search(index="offer_search", knn=retailer_query, _source=["OFFER","RETAILER"])["hits"]["hits"]

# Combine results from all three searches
combined_results = brand_hits + category_hits + retailer_hits

aggregated_results = {}
for hit in combined_results:
    doc_id = hit['_id']
    if doc_id not in aggregated_results:
        aggregated_results[doc_id] = {
            "score": hit['_score'],
            "source": hit['_source'],
            "count": 1
        }
    else:
        aggregated_results[doc_id]["score"] += hit['_score']
        aggregated_results[doc_id]["count"] += 1

# Normalize scores by count and sort the results
final_results = sorted(aggregated_results.values(), key=lambda x: x["score"] / x["count"], reverse=True)

final_results_filtered = [result for result in final_results if result['score'] >= 0.69]


# Output the final aggregated results
for result in final_results_filtered:
    print(result)


{'score': 1.0, 'source': {'OFFER': 'Beyond Steak™ Plant-Based seared tips, 10 ounce at Target', 'RETAILER': 'TARGET'}, 'count': 1}
{'score': 1.0, 'source': {'OFFER': 'Beyond Steak™ Plant-Based seared tips, 10 ounce at Target', 'RETAILER': 'TARGET'}, 'count': 1}


/var/folders/_r/rbpgvk997kx9h0kb4vvfpd3m0000gn/T/ipykernel_45106/3682287518.py:11: ElasticsearchWarning: The kNN search API has been replaced by the `knn` option in the search API.
  brand_hits = es.knn_search(index="offer_search", knn=brand_query, _source=["OFFER", 'BRAND'])["hits"]["hits"]
/var/folders/_r/rbpgvk997kx9h0kb4vvfpd3m0000gn/T/ipykernel_45106/3682287518.py:20: ElasticsearchWarning: The kNN search API has been replaced by the `knn` option in the search API.
  category_hits = es.knn_search(index="offer_search", knn=category_query, _source=["OFFER","CATEGORY"])["hits"]["hits"]
/var/folders/_r/rbpgvk997kx9h0kb4vvfpd3m0000gn/T/ipykernel_45106/3682287518.py:29: ElasticsearchWarning: The kNN search API has been replaced by the `knn` option in the search API.
  retailer_hits = es.knn_search(index="offer_search", knn=retailer_query, _source=["OFFER","RETAILER"])["hits"]["hits"]
